In [ ]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [ ]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

## Pyspark working with HBase


In [18]:
# Before running this demo make sure to 
# 1 - install package happybase (~$ sudo pip3 install happybase)
# 2 - start hbase and create stocks table (hbase> create 'stocks', 'info')
# 3 - Start hbase thrift service (~$ hbase-daemon.sh start thrift)
# Ref - https://blog.cloudera.com/how-to-use-the-hbase-thrift-interface-part-1/

import happybase

server = "127.0.0.1"
table_name = "stocks"
#table = happybase.Connection(server).table(table_name)
#table.put("r1", {"info:c1": "v1"})
#table.put("r1", {"info:c2": "v2"})
#table.put("r1", {"info:c3": "v3"})
# truncate 'stocks' table 
# (hbase> truncate 'stocks')

In [19]:
# Load stocks.csv in an RDD.

stocksRaw = sc.textFile("file:///home/talentum/test-jupyter/test/stocks.small.csv")

for r in stocksRaw.take(10):
    print(r)

date,open,high,low,close,volume,adjclose,symbol
2000-07-18,144.8125,144.828125,141.4375,143.0,5.06836E7,50.155473,INTC
2000-07-20,32.93751,34.25001,32.8125,33.75,3288300.0,8.789734,BEN
2000-07-24,64.25,67.312477,64.187523,64.75,948800.0,7.689567,APH
2000-07-26,21.875,22.125,20.9375,20.9375,1464300.0,15.61832,SHW
2000-07-26,42.0,42.312481,41.625,41.875,1397600.0,9.402721,STJ
2000-07-31,33.937519,33.999986,33.75,33.875011,273400.0,5.063753,GGP
2000-08-07,41.375038,43.0,40.812481,42.812481,2.43376E7,4.869542,SBUX
2000-08-09,55.562481,55.625,54.5,55.0,387600.0,10.495977,EQT
2000-08-10,49.0,52.625,48.75,52.125,1511600.0,22.539051,BCR


In [20]:
# Create a new RDD by filtering out the header.
stocks = stocksRaw.filter(lambda r: not r.startswith("date"))
for r in stocks.take(10):
    print(r)

2000-07-18,144.8125,144.828125,141.4375,143.0,5.06836E7,50.155473,INTC
2000-07-20,32.93751,34.25001,32.8125,33.75,3288300.0,8.789734,BEN
2000-07-24,64.25,67.312477,64.187523,64.75,948800.0,7.689567,APH
2000-07-26,21.875,22.125,20.9375,20.9375,1464300.0,15.61832,SHW
2000-07-26,42.0,42.312481,41.625,41.875,1397600.0,9.402721,STJ
2000-07-31,33.937519,33.999986,33.75,33.875011,273400.0,5.063753,GGP
2000-08-07,41.375038,43.0,40.812481,42.812481,2.43376E7,4.869542,SBUX
2000-08-09,55.562481,55.625,54.5,55.0,387600.0,10.495977,EQT
2000-08-10,49.0,52.625,48.75,52.125,1511600.0,22.539051,BCR
2000-08-15,35.68749,35.68749,34.875,34.875,1246500.0,9.082726,BEN


In [21]:
def bulk_insert(batch):
    table = happybase.Connection(server).table(table_name) 
    for r in batch:
        tokens = r.split(",")
        key = tokens[0] + "-" + tokens[7]
        value = {"info:date": tokens[0]
                 ,"info:open": tokens[1]
                 ,"info:high": tokens[2]
                 ,"info:low": tokens[3]
                 ,"info:close": tokens[4]
                 ,"info:volume": tokens[5]
                 ,"info:adjclose": tokens[6]
                 ,"info:symbol": tokens[7]
                }
        # Look at jupyter console to see the print output
        print(key, value)
        table.put(key, value)
   
stocks.foreachPartition(bulk_insert)

In [23]:
# hive on hbase
! pwd
! ./run-hive.sh

/home/talentum/test-jupyter/test
-----Creating Hive table on top of hbase----- 
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/talentum/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/talentum/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 2.3.6)
Driver: Hive JDBC (version 2.3.6)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/default> --drop table if exists stocks_hbase;
0: jdbc:hive2://localhost:10000/default> 
bname}.stocks_hbase(lhost:10000/default> CREATE EXTERNAL TABLE IF NOT EXISTS ${d 
. . . . . . . . . . . . . . . . . . . .>  key string,
